# Excercise A

Data la matrice, calcolarne la fattorizzazione lu di Gauss  facendo uso della funzione scipy.linalg.lu  e sfruttarla per il calcolo del suo determinante e della sua inversa.
$$
\left[
\begin{array}{cccc}
4.5 & 1 & 3 & 2\\
1 & -8 & 2 & 1\\
-1 & -2 & -3 & -1\\
2 & 6 & 0 & 1
\end{array}
\right ],
$$

# Excercise A1

Data La matrice, dire se esiste la fattorizzaione LU senza pivotaggio a perno massimo, verificando le ipotesi del teorema di esistenza della fattorizzazione LU

$$
\left[
\begin{array}{cccc}
4.5 & 1 & 3 & 2\\
1 & -8 & 2 & 1\\
-1 & -2 & -3 & -1\\
2 & 6 & 0 & 1
\end{array}
\right ],
$$

# Excercise A2
Data la matrice, dire se ammette fattorizzaione di Cholesky, verificando le ipotesi del teorema di Cholesky     
    
    A = np.array([[21., 37., 57.5], [ 37.    94.    72.5 ], [57.5, 72.5, 186.25]])

## Excercise B
 - Implementare la function  per il calcolo del polinomio interpolatore di Lagrange
 - Realizzare uno script python che interpola le coppie $x_i,y_i$, $i=0,\cdots,3$ dove $x_i,i=0,4$ sono punti equidistanti nell' intervallo [-2,2] e $y_i=f(x_i)$ dove $f(x)=5x^4+2x^2+2x-1$ 
 - Visualizzare il polinomio interpolatore, i nodi dell'interpolazione
 - Visualizzare l'errore assoluto tra il polinomio interpolatore e la funzione che ha generato i dati
 - Commentare i risultati alla luce della teoria.


# Excercise C
Per i dati $(x_i, y_i)$ riportati nei seguenti array
    
    x = np.array([0.0004, 0.2507, 0.5008, 2.0007, 8.0013]) 
    y = np.array([0.0007,0.0162, 0.0288, 0.0309, 0.0310])

 - costruire la retta di regressione;
 - costruire la parabola approssimante i dati nel senso dei minimi quadrati;
 - determinare l'approssimazione ai minimi quadrati espressa in termini di basi esponenziali: $y = a + b e^{-x}+ c e^{-2x}$

Quale tra le tre approssimazioni risulta la migliore? (Confrontare i grafici e la norma euclidea al quadrato del vettore dei residui).

# Excercise D
La temperatura T in prossimita' del suolo varia al variare della concentrazione k dell'acido carbonico e della latitudine L. Per k=1.5 la temperatura al suolo subisce una variazione dipendente dalla temperatura secondo la seguente tabella 

    L   [-55, -45, -35,	-25, -15, -5, 5, 15, 25, 35, 45, 55, 65]

    T	[3.7, 3., 3.52,	3.27, 3.2, 3.15, 3.15, 3.25, 3.47, 3.52, 3.65, 3.67, 3.52]

Si vuole costruire un MODELLO che descriva la legge T=T(L) anche per latitudini non misurate. 

Ad esempio si vuole valutare la variazione di temperatura a Londra ( L=51°). 
Mediante interpolazione con un polinomio di grado 12.
Valutare e disegnare, inoltre, il polinomio interpolatore di Lagrange in un set di 100 dati equidistanti nell’intervallo min(L), max(L).

Descrivere i risultati alla luce della teoria.

## Excercise E

 - Implementare il  metodo di Newton Raphson, la variante delle corde e la variante di Shamanskii per la soluzione di un sistema non lineare
Risolvere il sistema di equazioni non lineare, con ciascuno dei tre metodi e confrontare per ciascun metodo il grafico dell'errore relativo tra due iterati successivi, giustificando i risultati alla luce della teoria.

$$
\left[
\begin{array}{c}
x_0^2+x_0x_1=10 \\
 x_1+3x_0x_1^2=57
\end{array}
\right.
$$

 - Descrivi la variante del Metodod di Newton-Raphson per calcolare il minimo di una funzione non lineare in più variabili

**Nota Bene**: Servirsi del metodo grafico per individuare un 
iterato iniziale  $X_0$ nell'intorno della soluzione che si vuole approssimare. Visualizzare le 
curve di livello corrisponenti a z=O delle due superfici z = f1 (x1 ,x2 ) e z = f2(x1 , x 2 ) e 
definire come iterato iniziale un vettore $X_0$ oppurtuno le cui componenti stanno in un 
intorno della soluzione (punti di intersezione tra le curve di livello delle due superfici) \]